<a href="https://colab.research.google.com/github/kripet1/Datathon/blob/main/Copy_of_Untitled8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
from datasets import load_dataset
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Load dataset
dataset = load_dataset("rotten_tomatoes")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [ ]:
dataset

In [ ]:
# Access and print the first 10 rows of the training set
dataset["train"][:10]


In [ ]:
import re
import spacy
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense, Bidirectional
from keras.callbacks import EarlyStopping
import numpy as np

# Load spaCy model for English language
nlp = spacy.load("en_core_web_sm")

def advanced_preprocess(text):
    doc = nlp(text.lower())  # Process text with spaCy
    lemmatized = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(lemmatized)

# Assuming 'dataset' is your DatasetDict loaded from the 'rotten_tomatoes' dataset
processed_texts = [advanced_preprocess(text) for text in dataset['train']['text']]



In [ ]:
# Preprocess the dataset texts
def preprocess_dataset(part):
    return [advanced_preprocess(text) for text in part['text']], part['label']

train_texts, train_labels = preprocess_dataset(dataset['train'])
test_texts, test_labels = preprocess_dataset(dataset['test'])
val_texts, val_labels = preprocess_dataset(dataset['validation'])

# Tokenize texts
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)
word_index = tokenizer.word_index

# Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)

# Pad sequences
max_length = 100
padding_type='post'
trunc_type='post'

X_train = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
X_test = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
X_val = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

y_train = np.array(train_labels)
y_test = np.array(test_labels)
y_val = np.array(val_labels)

# Define the model
model = Sequential([
    Embedding(input_dim=10000, output_dim=16, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(24, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy}')


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 bidirectional (Bidirection  (None, 100, 128)          41472     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 24)                1560      
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 244273 (954.19 KB)
Trainable params: 24427

In [ ]:
from keras.layers import Dropout
from keras.optimizers import Adam

# Adjust the model architecture
model_improved = Sequential([
    Embedding(input_dim=10000, output_dim=32, input_length=max_length),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(64)),
    Dropout(0.5),
    Dense(24, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model with an adjusted learning rate
model_improved.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model_improved.summary()

# Train the improved model
history_improved = model_improved.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluate the improved model
test_loss_improved, test_accuracy_improved = model_improved.evaluate(X_test, y_test)
print(f'Test Accuracy (Improved): {test_accuracy_improved}')


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 32)           320000    
                                                                 
 bidirectional_2 (Bidirecti  (None, 100, 256)          164864    
 onal)                                                           
                                                                 
 dropout (Dropout)           (None, 100, 256)          0         
                                                                 
 bidirectional_3 (Bidirecti  (None, 128)               164352    
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 24)               

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Load the dataset
dataset = load_dataset("rotten_tomatoes")

# Advanced preprocessing using spaCy for lemmatization and stopword removal
import spacy
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    doc = nlp(text.lower())
    lemmatized = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.is_space]
    return " ".join(lemmatized)

# Apply preprocessing and split the data
X = [preprocess_text(text) for text in dataset['train']['text']]
y = np.array(dataset['train']['label'])

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Tokenize the text
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_length = max([len(x) for x in X_train_seq])

X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_val_padded = pad_sequences(X_val_seq, maxlen=max_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')

# Define the model with a new architecture incorporating CNN layers
model = Sequential([
    Embedding(input_dim=10000, output_dim=32, input_length=max_length),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(pool_size=4),
    LSTM(64, return_sequences=True),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Training the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(X_train_padded, y_train, epochs=10, validation_data=(X_val_padded, y_val), callbacks=[early_stopping])

# Evaluating the model
test_loss, test_acc = model.evaluate(X_test_padded, y_test)
print(f"Test Accuracy: {test_acc}")


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 34, 32)            320000    
                                                                 
 conv1d (Conv1D)             (None, 30, 64)            10304     
                                                                 
 max_pooling1d (MaxPooling1  (None, 7, 64)             0         
 D)                                                              
                                                                 
 lstm_4 (LSTM)               (None, 7, 64)             33024     
                                                                 
 global_max_pooling1d (Glob  (None, 64)                0         
 alMaxPooling1D)                                                 
                                                                 
 dense_4 (Dense)             (None, 64)               

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from datasets import load_dataset

# Load dataset
dataset = load_dataset("rotten_tomatoes")

# Preprocess texts and labels
X = dataset['train']['text']
y = dataset['train']['label']

# Split dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenization
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
max_length = max([len(x) for x in X_train_seq])
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length)
X_val_pad = pad_sequences(X_val_seq, maxlen=max_length)

# Prepare GloVe Embeddings
embedding_index = {}
glove_dir = './glove.6B/'
with open(('glove.6B.100d.txt'), encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

embedding_matrix = np.zeros((10000, 100))
for word, i in tokenizer.word_index.items():
    if i < 10000:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Define Model
model = Sequential([
    Embedding(10000, 100, input_length=max_length, weights=[embedding_matrix], trainable=False),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(pool_size=4),
    LSTM(64, return_sequences=True),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train Model
model.fit(X_train_pad, np.array(y_train), batch_size=32, epochs=10, validation_data=(X_val_pad, np.array(y_val)))

# Evaluate Model
# Note: Make sure to preprocess your test dataset and use model.evaluate on your test data.
# Preprocess the test data
X_test = dataset['test']['text']
y_test = dataset['test']['label']

X_test_seq = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test_pad, np.array(y_test), verbose=2)
print('Test Accuracy:', test_acc)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 51, 100)           1000000   
                                                                 
 conv1d_1 (Conv1D)           (None, 47, 64)            32064     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 11, 64)            0         
 g1D)                                                            
                                                                 
 lstm_5 (LSTM)               (None, 11, 64)            33024     
                                                                 
 global_max_pooling1d_1 (Gl  (None, 64)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_6 (Dense)             (None, 64)               

er

In [ ]:
import numpy as np
from datasets import load_dataset
import spacy
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    doc = nlp(text.lower())
    lemmatized = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.is_space]
    return " ".join(lemmatized)

dataset = load_dataset("rotten_tomatoes")
X = [preprocess_text(text) for text in dataset['train']['text']]
y = np.array(dataset['train']['label'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_length = max(max(len(x) for x in X_train_seq), max(len(x) for x in X_test_seq))

X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')


In [ ]:
embedding_index = {}
with open('glove.6B.100d.txt', 'r', encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

embedding_dim = 100
embedding_matrix = np.zeros((tokenizer.num_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < tokenizer.num_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout

model = Sequential([
    Embedding(tokenizer.num_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False),
    Bidirectional(LSTM(128, return_sequences=True)),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 34, 100)           1000000   
                                                                 
 bidirectional_4 (Bidirecti  (None, 34, 256)           234496    
 onal)                                                           
                                                                 
 bidirectional_5 (Bidirecti  (None, 128)               164352    
 onal)                                                           
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 1)                

In [ ]:
history = model.fit(X_train_pad, y_train, batch_size=32, epochs=10, validation_split=0.2)


Epoch 1/10
171/171 [==============================] - 44s 210ms/step - loss: 0.6160 - accuracy: 0.6661 - val_loss: 0.5726 - val_accuracy: 0.6945
Epoch 2/10
171/171 [==============================] - 33s 193ms/step - loss: 0.5532 - accuracy: 0.7192 - val_loss: 0.6037 - val_accuracy: 0.6835
Epoch 3/10
171/171 [==============================] - 33s 190ms/step - loss: 0.5256 - accuracy: 0.7397 - val_loss: 0.5584 - val_accuracy: 0.7128
Epoch 4/10
171/171 [==============================] - 31s 184ms/step - loss: 0.4966 - accuracy: 0.7569 - val_loss: 0.5676 - val_accuracy: 0.7165
Epoch 5/10
171/171 [==============================] - 36s 210ms/step - loss: 0.4686 - accuracy: 0.7707 - val_loss: 0.5539 - val_accuracy: 0.7223
Epoch 6/10
171/171 [==============================] - 31s 181ms/step - loss: 0.4305 - accuracy: 0.7998 - val_loss: 0.5682 - val_accuracy: 0.7187
Epoch 7/10
171/171 [==============================] - 31s 181ms/step - loss: 0.3771 - accuracy: 0.8267 - val_loss: 0.6120 - val_ac

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_pad, y_test)
print(f'Test Accuracy: {test_accuracy}')


54/54 [==============================] - 3s 64ms/step - loss: 0.8123 - accuracy: 0.7087
Test Accuracy: 0.7086752653121948
